# Phase 1: Data Audit & EDA for Research

**Goal:** Understand exactly what data we have, its quality, and prepare for RQ1 experiments  
**Research Question 1:** Does decomposing environmental prediction into specialized agents outperform monolithic models?

---

## Datasets:
1. `ml_ready_data.csv` — 53K rows, 10 features (pre-processed)
2. `in_situ/LUBW_BW_measurements...csv` — 49K rows, 23 features (raw in-situ)
3. `satellite/*.csv` — 8.7K rows (spectral indices)
4. `lake_data/*.json` — yearly lake data with bloom indicators

## Outputs:
- Data quality report
- Clean, merged research dataset
- Bloom labels (binary + continuous)
- Train/val/test splits (temporal)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import glob
import os
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Project paths
PROJECT_ROOT = Path('/Users/futurediary/Desktop/A.I. Agents/ERAY_HEIDELBERG')
DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
HARMONIZED_DIR = DATA_DIR / 'harmonized'

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11

print('Setup complete.')

---
## 1. Load All Datasets

In [ ]:
# 1A: ML-ready data
ml_data = pd.read_csv(PROCESSED_DIR / 'ml_ready_data.csv', parse_dates=['timestamp'])
print(f'ml_ready_data: {ml_data.shape[0]:,} rows x {ml_data.shape[1]} cols')
print(f'  Date range: {ml_data["timestamp"].min()} → {ml_data["timestamp"].max()}')
print(f'  Lakes: {ml_data["lake"].nunique()} unique → {ml_data["lake"].value_counts().head(10).to_dict()}')
print()

In [ ]:
# 1B: In-situ measurements
insitu = pd.read_csv(RAW_DIR / 'in_situ' / 'LUBW_BW_measurements_Seen_Wasserqualitaet_Extracted.csv',
                     parse_dates=['timestamp'])
print(f'in_situ: {insitu.shape[0]:,} rows x {insitu.shape[1]} cols')
print(f'  Date range: {insitu["timestamp"].min()} → {insitu["timestamp"].max()}')
print(f'  Lakes: {insitu["location_name"].nunique()} unique')
print(f'  Columns: {list(insitu.columns)}')
print()

In [ ]:
# 1C: Satellite data (combine all CSVs)
sat_files = sorted(glob.glob(str(RAW_DIR / 'satellite' / '*.csv')))
sat_dfs = []
for f in sat_files:
    df = pd.read_csv(f, parse_dates=['acquisition_date'])
    df['source_file'] = Path(f).name
    sat_dfs.append(df)
    print(f'  {Path(f).name}: {len(df):,} rows')

satellite = pd.concat(sat_dfs, ignore_index=True)
print(f'\nSatellite combined: {satellite.shape[0]:,} rows x {satellite.shape[1]} cols')
print(f'  Date range: {satellite["acquisition_date"].min()} → {satellite["acquisition_date"].max()}')
print(f'  Lakes: {satellite["lake_name"].nunique()} → {satellite["lake_name"].value_counts().to_dict()}')
print()

In [ ]:
# 1D: Lake JSON data (has bloom labels!)
lake_json_files = sorted(glob.glob(str(RAW_DIR / 'lake_data' / '*.json')))
lake_records = []
for f in lake_json_files:
    if 'current_week' not in f:
        with open(f) as fh:
            data = json.load(fh)
            for rec in data:
                rec['source_file'] = Path(f).name
            lake_records.extend(data)
            print(f'  {Path(f).name}: {len(data)} records')

# Flatten nested JSON
lake_flat = []
for rec in lake_records:
    flat = {
        'date': rec.get('date'),
        'lake': rec.get('lake'),
        'source': rec.get('source'),
        'quality_score': rec.get('quality_score'),
        'source_file': rec.get('source_file'),
    }
    # Flatten parameters
    for k, v in rec.get('parameters', {}).items():
        flat[f'param_{k}'] = v
    # Flatten HAB indicators
    for k, v in rec.get('habs_indicators', {}).items():
        flat[f'hab_{k}'] = v
    lake_flat.append(flat)

lake_data = pd.DataFrame(lake_flat)
lake_data['date'] = pd.to_datetime(lake_data['date'])
print(f'\nLake JSON combined: {lake_data.shape[0]:,} rows')
print(f'  Date range: {lake_data["date"].min()} → {lake_data["date"].max()}')
print(f'  Lakes: {lake_data["lake"].nunique()} → {lake_data["lake"].value_counts().to_dict()}')
print(f'  Bloom statuses: {lake_data["hab_bloom_status"].value_counts().to_dict()}')

---
## 2. Data Quality Audit
Check for synthetic data, missing values, suspicious patterns

In [ ]:
# 2A: ML-ready data quality
print('=' * 60)
print('ML-READY DATA QUALITY AUDIT')
print('=' * 60)

feature_cols = ['chlorophyll_a', 'water_temperature', 'turbidity', 'dissolved_oxygen',
                'ph', 'total_nitrogen', 'total_phosphorus', 'solar_radiation',
                'wind_speed', 'precipitation']

for col in feature_cols:
    zeros = (ml_data[col] == 0).sum()
    nulls = ml_data[col].isna().sum()
    unique = ml_data[col].nunique()
    flag = '⚠️ SUSPICIOUS' if zeros / len(ml_data) > 0.5 else '✓'
    print(f'  {col:25s} | zeros: {zeros:6d} ({zeros/len(ml_data)*100:5.1f}%) | nulls: {nulls:5d} | unique: {unique:6d} | {flag}')

# Check lake name quality
print(f'\n  Lake column issues:')
print(f'    Empty/null: {ml_data["lake"].isna().sum()}')
print(f'    "object" as lake name: {(ml_data["lake"] == "object").sum()}')
print(f'    Unique valid lakes: {ml_data[ml_data["lake"] != "object"]["lake"].nunique()}')

In [ ]:
# 2B: In-situ data quality
print('=' * 60)
print('IN-SITU DATA QUALITY AUDIT')
print('=' * 60)

insitu_features = ['chlorophyll_a', 'turbidity', 'dissolved_oxygen', 'ph',
                   'conductivity', 'temperature', 'wind_speed', 'air_temperature', 'humidity']

for col in insitu_features:
    if col in insitu.columns:
        zeros = (insitu[col] == 0).sum()
        nulls = insitu[col].isna().sum()
        mn, mx = insitu[col].min(), insitu[col].max()
        print(f'  {col:25s} | range: [{mn:.2f}, {mx:.2f}] | zeros: {zeros:5d} | nulls: {nulls:5d}')

print(f'\n  Data sources: {insitu["data_source"].value_counts().to_dict()}')
print(f'  Locations: {insitu["location_name"].value_counts().head(10).to_dict()}')

In [ ]:
# 2C: Cross-dataset consistency check
print('=' * 60)
print('CROSS-DATASET CONSISTENCY')
print('=' * 60)

# Compare chlorophyll_a distributions across sources
fig, axes = plt.subplots(1, 4, figsize=(20, 5))

axes[0].hist(ml_data['chlorophyll_a'].dropna(), bins=50, alpha=0.7, color='steelblue')
axes[0].set_title(f'ml_ready: chlorophyll_a\nμ={ml_data["chlorophyll_a"].mean():.2f}, σ={ml_data["chlorophyll_a"].std():.2f}')
axes[0].set_xlabel('chlorophyll_a (μg/L)')

axes[1].hist(insitu['chlorophyll_a'].dropna(), bins=50, alpha=0.7, color='coral')
axes[1].set_title(f'in_situ: chlorophyll_a\nμ={insitu["chlorophyll_a"].mean():.2f}, σ={insitu["chlorophyll_a"].std():.2f}')
axes[1].set_xlabel('chlorophyll_a (μg/L)')

axes[2].hist(satellite['chlorophyll_index'].dropna(), bins=50, alpha=0.7, color='seagreen')
axes[2].set_title(f'satellite: chlorophyll_index\nμ={satellite["chlorophyll_index"].mean():.2f}, σ={satellite["chlorophyll_index"].std():.2f}')
axes[2].set_xlabel('chlorophyll_index')

axes[3].hist(lake_data['param_chlorophyll_a'].dropna(), bins=50, alpha=0.7, color='orchid')
axes[3].set_title(f'lake_json: chlorophyll_a\nμ={lake_data["param_chlorophyll_a"].mean():.2f}, σ={lake_data["param_chlorophyll_a"].std():.2f}')
axes[3].set_xlabel('chlorophyll_a (μg/L)')

plt.suptitle('Chlorophyll-a Distribution Across All Data Sources', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---
## 3. Temporal Coverage Analysis
Which lakes have data, for which time periods?

In [ ]:
# 3A: Temporal heatmap - records per lake per month
fig, axes = plt.subplots(2, 2, figsize=(18, 12))

# ML-ready data
ml_valid = ml_data[ml_data['lake'] != 'object'].copy()
if len(ml_valid) > 0:
    ml_valid['month'] = ml_valid['timestamp'].dt.to_period('M').astype(str)
    pivot = ml_valid.groupby(['lake', 'month']).size().unstack(fill_value=0)
    sns.heatmap(pivot, ax=axes[0, 0], cmap='YlOrRd', cbar_kws={'label': 'Count'})
    axes[0, 0].set_title('ML-Ready Data: Records per Lake/Month')
    axes[0, 0].tick_params(axis='x', rotation=45)

# In-situ data
insitu_copy = insitu.copy()
insitu_copy['month'] = insitu_copy['timestamp'].dt.to_period('M').astype(str)
pivot2 = insitu_copy.groupby(['location_name', 'month']).size().unstack(fill_value=0)
sns.heatmap(pivot2, ax=axes[0, 1], cmap='YlOrRd', cbar_kws={'label': 'Count'})
axes[0, 1].set_title('In-Situ: Records per Location/Month')
axes[0, 1].tick_params(axis='x', rotation=45)

# Satellite data
sat_copy = satellite.copy()
sat_copy['month'] = sat_copy['acquisition_date'].dt.to_period('M').astype(str)
pivot3 = sat_copy.groupby(['lake_name', 'month']).size().unstack(fill_value=0)
sns.heatmap(pivot3, ax=axes[1, 0], cmap='YlOrRd', cbar_kws={'label': 'Count'})
axes[1, 0].set_title('Satellite: Records per Lake/Month')
axes[1, 0].tick_params(axis='x', rotation=45)

# Lake JSON data
lake_copy = lake_data.copy()
lake_copy['month'] = lake_copy['date'].dt.to_period('M').astype(str)
pivot4 = lake_copy.groupby(['lake', 'month']).size().unstack(fill_value=0)
sns.heatmap(pivot4, ax=axes[1, 1], cmap='YlOrRd', cbar_kws={'label': 'Count'})
axes[1, 1].set_title('Lake JSON: Records per Lake/Month')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.suptitle('Temporal Coverage Across All Data Sources', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---
## 4. Feature Distributions & Correlations

In [ ]:
# 4A: Feature distributions for in-situ data (richest source)
insitu_numeric = insitu[['chlorophyll_a', 'turbidity', 'dissolved_oxygen', 'ph',
                         'temperature', 'conductivity', 'wind_speed', 'air_temperature', 'humidity']].copy()

fig, axes = plt.subplots(3, 3, figsize=(16, 12))
for i, col in enumerate(insitu_numeric.columns):
    ax = axes[i // 3, i % 3]
    insitu_numeric[col].dropna().hist(bins=50, ax=ax, alpha=0.7, color='steelblue', edgecolor='white')
    ax.set_title(f'{col}\nμ={insitu_numeric[col].mean():.2f}, σ={insitu_numeric[col].std():.2f}')
    ax.axvline(insitu_numeric[col].median(), color='red', linestyle='--', alpha=0.7, label='median')

plt.suptitle('In-Situ Feature Distributions', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# 4B: Correlation matrix
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# In-situ correlations
corr1 = insitu_numeric.corr()
mask1 = np.triu(np.ones_like(corr1, dtype=bool))
sns.heatmap(corr1, mask=mask1, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
            ax=axes[0], vmin=-1, vmax=1, square=True)
axes[0].set_title('In-Situ Feature Correlations')

# ML-ready correlations
ml_numeric = ml_data[feature_cols].copy()
corr2 = ml_numeric.corr()
mask2 = np.triu(np.ones_like(corr2, dtype=bool))
sns.heatmap(corr2, mask=mask2, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
            ax=axes[1], vmin=-1, vmax=1, square=True)
axes[1].set_title('ML-Ready Feature Correlations')

plt.suptitle('Feature Correlation Comparison', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---
## 5. Bloom Label Analysis
Examine existing bloom indicators from lake JSON data & create labels

In [ ]:
# 5A: Bloom indicators from lake JSON
print('=' * 60)
print('BLOOM LABEL ANALYSIS (Lake JSON Data)')
print('=' * 60)

print(f'\nBloom probability distribution:')
print(lake_data['hab_bloom_probability'].describe())

print(f'\nBloom status distribution:')
print(lake_data['hab_bloom_status'].value_counts())

print(f'\nCyanobacteria density distribution:')
print(lake_data['hab_cyanobacteria_density'].describe())

print(f'\nToxin levels distribution:')
print(lake_data['hab_toxin_levels'].describe())

In [ ]:
# 5B: Visualize bloom distributions
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Bloom probability
lake_data['hab_bloom_probability'].hist(bins=30, ax=axes[0, 0], color='crimson', alpha=0.7, edgecolor='white')
axes[0, 0].set_title('Bloom Probability Distribution')
axes[0, 0].axvline(0.5, color='black', linestyle='--', label='threshold=0.5')
axes[0, 0].legend()

# Bloom status
lake_data['hab_bloom_status'].value_counts().plot.bar(ax=axes[0, 1], color='steelblue', edgecolor='white')
axes[0, 1].set_title('Bloom Status Counts')
axes[0, 1].tick_params(axis='x', rotation=45)

# Cyanobacteria density
lake_data['hab_cyanobacteria_density'].hist(bins=30, ax=axes[1, 0], color='seagreen', alpha=0.7, edgecolor='white')
axes[1, 0].set_title('Cyanobacteria Density')

# Bloom probability by lake
lake_data.boxplot(column='hab_bloom_probability', by='lake', ax=axes[1, 1])
axes[1, 1].set_title('Bloom Probability by Lake')
axes[1, 1].tick_params(axis='x', rotation=45)
plt.suptitle('')  # remove default title

plt.suptitle('Bloom Indicator Analysis', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# 5C: Create binary bloom labels using multiple strategies
print('=' * 60)
print('LABEL STRATEGY COMPARISON')
print('=' * 60)

# Strategy 1: Threshold on bloom_probability (from lake JSON)
lake_data['label_prob50'] = (lake_data['hab_bloom_probability'] >= 0.5).astype(int)
lake_data['label_prob30'] = (lake_data['hab_bloom_probability'] >= 0.3).astype(int)

# Strategy 2: Bloom status (categorical)
status_map = {'None': 0, 'Low': 0, 'Moderate': 1, 'High': 1, 'Critical': 1}
lake_data['label_status'] = lake_data['hab_bloom_status'].map(status_map).fillna(0).astype(int)

# Strategy 3: Chlorophyll threshold (standard approach)
lake_data['label_chl20'] = (lake_data['param_chlorophyll_a'] > 20).astype(int)
lake_data['label_chl10'] = (lake_data['param_chlorophyll_a'] > 10).astype(int)

# Compare label strategies
label_cols = ['label_prob50', 'label_prob30', 'label_status', 'label_chl20', 'label_chl10']
for col in label_cols:
    pos = lake_data[col].sum()
    total = len(lake_data)
    print(f'  {col:20s} | positive: {pos:5d}/{total} ({pos/total*100:5.1f}%) | ratio: 1:{total//max(pos,1)}')

---
## 6. Build Unified Research Dataset
Merge in-situ + satellite + bloom labels into one clean dataset

In [ ]:
# 6A: Prepare in-situ data as primary source
# It has the richest feature set (23 cols)
research_insitu = insitu[['location_name', 'timestamp', 'latitude', 'longitude', 'station_id',
                           'chlorophyll_a', 'turbidity', 'dissolved_oxygen', 'ph',
                           'temperature', 'conductivity', 'wind_speed', 'air_temperature',
                           'humidity', 'quality_score', 'data_source', 'notes']].copy()

# Standardize lake names for merging
research_insitu['lake'] = research_insitu['location_name'].str.split(' - ').str[0].str.strip()
research_insitu['date'] = research_insitu['timestamp'].dt.date

print(f'In-situ prepared: {len(research_insitu):,} rows')
print(f'Lakes: {research_insitu["lake"].value_counts().to_dict()}')

In [ ]:
# 6B: Aggregate to daily level per lake (multiple measurements per day)
daily_insitu = research_insitu.groupby(['lake', 'date']).agg({
    'chlorophyll_a': 'mean',
    'turbidity': 'mean',
    'dissolved_oxygen': 'mean',
    'ph': 'mean',
    'temperature': 'mean',
    'conductivity': 'mean',
    'wind_speed': 'mean',
    'air_temperature': 'mean',
    'humidity': 'mean',
    'latitude': 'first',
    'longitude': 'first',
    'quality_score': 'mean',
}).reset_index()

daily_insitu['date'] = pd.to_datetime(daily_insitu['date'])
print(f'Daily aggregated: {len(daily_insitu):,} rows')
print(daily_insitu.head())

In [ ]:
# 6C: Merge satellite data (nearest date per lake)
# Prepare satellite features
sat_features = satellite[['lake_name', 'acquisition_date', 'ndvi', 'surface_temperature',
                           'chlorophyll_index', 'turbidity_index', 'cloud_coverage',
                           'reflectance_blue', 'reflectance_green', 'reflectance_red',
                           'reflectance_nir']].copy()
sat_features = sat_features.rename(columns={'lake_name': 'lake', 'acquisition_date': 'date'})
sat_features['date'] = sat_features['date'].dt.normalize()

# Daily aggregate for satellite
daily_sat = sat_features.groupby(['lake', 'date']).agg({
    'ndvi': 'mean',
    'surface_temperature': 'mean',
    'chlorophyll_index': 'mean',
    'turbidity_index': 'mean',
    'cloud_coverage': 'mean',
    'reflectance_blue': 'mean',
    'reflectance_green': 'mean',
    'reflectance_red': 'mean',
    'reflectance_nir': 'mean',
}).reset_index()

print(f'Daily satellite: {len(daily_sat):,} rows')
print(f'Lakes: {daily_sat["lake"].value_counts().to_dict()}')

In [ ]:
# 6D: Merge bloom labels from lake JSON
lake_labels = lake_data[['date', 'lake', 'hab_bloom_probability', 'hab_bloom_status',
                          'hab_cyanobacteria_density', 'hab_toxin_levels',
                          'param_chlorophyll_a']].copy()
lake_labels['date'] = lake_labels['date'].dt.normalize()

# Daily aggregate
daily_labels = lake_labels.groupby(['lake', 'date']).agg({
    'hab_bloom_probability': 'mean',
    'hab_cyanobacteria_density': 'mean',
    'hab_toxin_levels': 'mean',
    'hab_bloom_status': 'first',
}).reset_index()

print(f'Daily labels: {len(daily_labels):,} rows')
print(f'Lakes: {daily_labels["lake"].value_counts().to_dict()}')

In [ ]:
# 6E: Merge everything
# Start with in-situ as base, merge satellite and labels
research_df = daily_insitu.copy()

# Merge satellite (on lake + date, allowing some date tolerance with merge_asof)
research_df = research_df.sort_values('date')
daily_sat_sorted = daily_sat.sort_values('date')

merged_parts = []
for lake in research_df['lake'].unique():
    left = research_df[research_df['lake'] == lake].copy()
    right = daily_sat_sorted[daily_sat_sorted['lake'] == lake].copy()
    if len(right) > 0:
        merged = pd.merge_asof(left, right.drop(columns=['lake']),
                               on='date', tolerance=pd.Timedelta('3D'),
                               direction='nearest')
    else:
        merged = left
    merged_parts.append(merged)

research_df = pd.concat(merged_parts, ignore_index=True)

# Merge bloom labels
daily_labels_sorted = daily_labels.sort_values('date')
merged_parts2 = []
for lake in research_df['lake'].unique():
    left = research_df[research_df['lake'] == lake].sort_values('date').copy()
    right = daily_labels_sorted[daily_labels_sorted['lake'] == lake].copy()
    if len(right) > 0:
        merged = pd.merge_asof(left, right.drop(columns=['lake']),
                               on='date', tolerance=pd.Timedelta('7D'),
                               direction='nearest')
    else:
        merged = left
    merged_parts2.append(merged)

research_df = pd.concat(merged_parts2, ignore_index=True)

print(f'\n{"=" * 60}')
print(f'UNIFIED RESEARCH DATASET')
print(f'{"=" * 60}')
print(f'Shape: {research_df.shape}')
print(f'Columns: {list(research_df.columns)}')
print(f'Lakes: {research_df["lake"].value_counts().to_dict()}')
print(f'Date range: {research_df["date"].min()} → {research_df["date"].max()}')
print(f'\nNull counts:')
print(research_df.isnull().sum().sort_values(ascending=False).head(15))

---
## 7. Create Bloom Labels for Research

In [ ]:
# 7A: Binary bloom label — use bloom_probability if available, else chlorophyll threshold
def create_bloom_label(row, threshold=0.5):
    """Multi-criteria bloom label."""
    # Priority 1: Use bloom_probability from monitoring data
    if pd.notna(row.get('hab_bloom_probability')):
        return int(row['hab_bloom_probability'] >= threshold)
    # Priority 2: Chlorophyll-based threshold (WHO guideline: 10 μg/L moderate, 50 μg/L high)
    chl = row.get('chlorophyll_a', np.nan)
    temp = row.get('temperature', np.nan)
    if pd.notna(chl):
        if chl > 20:
            return 1
        if chl > 10 and pd.notna(temp) and temp > 18:
            return 1
    return 0

research_df['bloom_label'] = research_df.apply(create_bloom_label, axis=1)

# Continuous target (for regression)
research_df['bloom_risk'] = research_df['hab_bloom_probability'].fillna(
    research_df['chlorophyll_a'].clip(0, 50) / 50  # normalize chl to 0-1 as proxy
)

pos = research_df['bloom_label'].sum()
total = len(research_df)
print(f'Bloom label distribution: {pos} positive / {total} total ({pos/total*100:.1f}%)')
print(f'Bloom risk (continuous): mean={research_df["bloom_risk"].mean():.3f}, std={research_df["bloom_risk"].std():.3f}')

In [ ]:
# 7B: Visualize label quality
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Label balance
research_df['bloom_label'].value_counts().plot.bar(ax=axes[0], color=['steelblue', 'crimson'], edgecolor='white')
axes[0].set_title('Binary Label Balance')
axes[0].set_xticklabels(['No Bloom (0)', 'Bloom (1)'], rotation=0)

# Risk distribution
research_df['bloom_risk'].hist(bins=30, ax=axes[1], color='orchid', alpha=0.7, edgecolor='white')
axes[1].set_title('Continuous Bloom Risk Distribution')
axes[1].axvline(0.5, color='black', linestyle='--')

# Bloom rate per lake
bloom_by_lake = research_df.groupby('lake')['bloom_label'].mean().sort_values(ascending=False)
bloom_by_lake.plot.bar(ax=axes[2], color='seagreen', edgecolor='white')
axes[2].set_title('Bloom Rate by Lake')
axes[2].set_ylabel('Fraction with bloom=1')
axes[2].tick_params(axis='x', rotation=45)

plt.suptitle('Bloom Label Quality Assessment', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---
## 8. Train/Val/Test Split (Temporal)
**Critical**: Use temporal split, NOT random, to avoid data leakage

In [ ]:
# 8A: Temporal split
research_df = research_df.sort_values('date').reset_index(drop=True)

# Determine split points based on actual date range
date_range = research_df['date'].max() - research_df['date'].min()
train_end = research_df['date'].min() + pd.Timedelta(days=int(date_range.days * 0.7))
val_end = research_df['date'].min() + pd.Timedelta(days=int(date_range.days * 0.85))

train = research_df[research_df['date'] <= train_end]
val = research_df[(research_df['date'] > train_end) & (research_df['date'] <= val_end)]
test = research_df[research_df['date'] > val_end]

print(f'Temporal Split:')
print(f'  Train: {len(train):,} rows | {train["date"].min()} → {train["date"].max()} | bloom rate: {train["bloom_label"].mean():.3f}')
print(f'  Val:   {len(val):,} rows | {val["date"].min()} → {val["date"].max()} | bloom rate: {val["bloom_label"].mean():.3f}')
print(f'  Test:  {len(test):,} rows | {test["date"].min()} → {test["date"].max()} | bloom rate: {test["bloom_label"].mean():.3f}')

In [ ]:
# 8B: Visualize the split
fig, ax = plt.subplots(figsize=(16, 4))

ax.scatter(train['date'], [0]*len(train), alpha=0.3, s=5, c='steelblue', label=f'Train ({len(train):,})')
ax.scatter(val['date'], [1]*len(val), alpha=0.3, s=5, c='orange', label=f'Val ({len(val):,})')
ax.scatter(test['date'], [2]*len(test), alpha=0.3, s=5, c='crimson', label=f'Test ({len(test):,})')

ax.axvline(train_end, color='black', linestyle='--', alpha=0.5)
ax.axvline(val_end, color='black', linestyle='--', alpha=0.5)
ax.set_yticks([0, 1, 2])
ax.set_yticklabels(['Train', 'Val', 'Test'])
ax.set_title('Temporal Train/Val/Test Split')
ax.legend()
plt.tight_layout()
plt.show()

---
## 9. Save Research Dataset

In [ ]:
# 9A: Save the unified research dataset
output_dir = DATA_DIR / 'research'
output_dir.mkdir(exist_ok=True)

research_df.to_parquet(output_dir / 'unified_research_dataset.parquet', index=False)
train.to_parquet(output_dir / 'train.parquet', index=False)
val.to_parquet(output_dir / 'val.parquet', index=False)
test.to_parquet(output_dir / 'test.parquet', index=False)

# Save metadata
metadata = {
    'created': datetime.now().isoformat(),
    'total_records': len(research_df),
    'train_records': len(train),
    'val_records': len(val),
    'test_records': len(test),
    'train_end': str(train_end),
    'val_end': str(val_end),
    'lakes': research_df['lake'].unique().tolist(),
    'features_insitu': ['chlorophyll_a', 'turbidity', 'dissolved_oxygen', 'ph',
                        'temperature', 'conductivity', 'wind_speed', 'air_temperature', 'humidity'],
    'features_satellite': ['ndvi', 'surface_temperature', 'chlorophyll_index',
                           'turbidity_index', 'cloud_coverage',
                           'reflectance_blue', 'reflectance_green',
                           'reflectance_red', 'reflectance_nir'],
    'label_binary': 'bloom_label',
    'label_continuous': 'bloom_risk',
    'bloom_positive_rate': float(research_df['bloom_label'].mean()),
}

with open(output_dir / 'dataset_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2, default=str)

print(f'Saved to {output_dir}/')
print(f'  unified_research_dataset.parquet: {len(research_df):,} rows')
print(f'  train.parquet: {len(train):,} rows')
print(f'  val.parquet: {len(val):,} rows')
print(f'  test.parquet: {len(test):,} rows')
print(f'  dataset_metadata.json')

---
## 10. Quick Baseline: XGBoost (Monolithic)
First baseline for RQ1 — single model using ALL available features

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import (roc_auc_score, average_precision_score, brier_score_loss,
                             classification_report, confusion_matrix, roc_curve, precision_recall_curve)
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import pickle

# Define feature sets for experiment
INSITU_FEATURES = ['chlorophyll_a', 'turbidity', 'dissolved_oxygen', 'ph',
                   'temperature', 'conductivity', 'wind_speed', 'air_temperature', 'humidity']

SATELLITE_FEATURES = ['ndvi', 'surface_temperature', 'chlorophyll_index',
                      'turbidity_index', 'cloud_coverage']

ALL_FEATURES = INSITU_FEATURES + SATELLITE_FEATURES

TARGET = 'bloom_label'

print(f'In-situ features ({len(INSITU_FEATURES)}): {INSITU_FEATURES}')
print(f'Satellite features ({len(SATELLITE_FEATURES)}): {SATELLITE_FEATURES}')
print(f'All features ({len(ALL_FEATURES)}): {ALL_FEATURES}')

In [ ]:
# 10A: Prepare train/val/test matrices
def prepare_Xy(df, features, target):
    """Extract feature matrix and labels, handling missing values."""
    available = [f for f in features if f in df.columns]
    X = df[available].copy()
    y = df[target].copy()
    return X, y, available

X_train, y_train, used_features = prepare_Xy(train, ALL_FEATURES, TARGET)
X_val, y_val, _ = prepare_Xy(val, used_features, TARGET)
X_test, y_test, _ = prepare_Xy(test, used_features, TARGET)

# Impute + Scale
imputer = SimpleImputer(strategy='median')
scaler = StandardScaler()

X_train_imp = imputer.fit_transform(X_train)
X_val_imp = imputer.transform(X_val)
X_test_imp = imputer.transform(X_test)

X_train_scaled = scaler.fit_transform(X_train_imp)
X_val_scaled = scaler.transform(X_val_imp)
X_test_scaled = scaler.transform(X_test_imp)

print(f'Features used ({len(used_features)}): {used_features}')
print(f'Train: X={X_train_scaled.shape}, y={y_train.shape} (bloom rate: {y_train.mean():.3f})')
print(f'Val:   X={X_val_scaled.shape}, y={y_val.shape} (bloom rate: {y_val.mean():.3f})')
print(f'Test:  X={X_test_scaled.shape}, y={y_test.shape} (bloom rate: {y_test.mean():.3f})')

In [ ]:
# 10B: Train baseline models
results = {}

# Model 1: Gradient Boosting (XGBoost-like)
gb = GradientBoostingClassifier(n_estimators=200, max_depth=5, learning_rate=0.1,
                                 subsample=0.8, random_state=42)
gb.fit(X_train_scaled, y_train)

# Model 2: Random Forest
rf = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42, n_jobs=-1)
rf.fit(X_train_scaled, y_train)

# Model 3: Ensemble (average of GB + RF)
models = {
    'GradientBoosting': gb,
    'RandomForest': rf,
}

print('Models trained.')
for name, model in models.items():
    print(f'  {name}: {model.n_estimators} estimators')

In [ ]:
# 10C: Evaluate on validation set
def evaluate_model(name, y_true, y_prob):
    """Compute research-relevant metrics."""
    y_pred = (y_prob >= 0.5).astype(int)
    metrics = {
        'AUROC': roc_auc_score(y_true, y_prob) if len(np.unique(y_true)) > 1 else float('nan'),
        'AUPRC': average_precision_score(y_true, y_prob) if len(np.unique(y_true)) > 1 else float('nan'),
        'Brier': brier_score_loss(y_true, y_prob),
        'Accuracy': (y_pred == y_true).mean(),
    }
    return metrics

print(f'{"Model":<25s} {"AUROC":>8s} {"AUPRC":>8s} {"Brier":>8s} {"Acc":>8s}')
print('-' * 60)

ensemble_probs_val = np.zeros(len(y_val))
ensemble_probs_test = np.zeros(len(y_test))

for name, model in models.items():
    y_prob_val = model.predict_proba(X_val_scaled)[:, 1]
    y_prob_test = model.predict_proba(X_test_scaled)[:, 1]
    ensemble_probs_val += y_prob_val / len(models)
    ensemble_probs_test += y_prob_test / len(models)

    m = evaluate_model(name, y_val, y_prob_val)
    results[name] = m
    print(f'{name:<25s} {m["AUROC"]:>8.4f} {m["AUPRC"]:>8.4f} {m["Brier"]:>8.4f} {m["Acc"]:>8.4f}')

# Ensemble
m_ens = evaluate_model('Ensemble', y_val, ensemble_probs_val)
results['Ensemble_GB_RF'] = m_ens
print(f'{"Ensemble (GB+RF)":<25s} {m_ens["AUROC"]:>8.4f} {m_ens["AUPRC"]:>8.4f} {m_ens["Brier"]:>8.4f} {m_ens["Acc"]:>8.4f}')

In [ ]:
# 10D: Test set evaluation (final numbers)
print('\n' + '=' * 60)
print('TEST SET RESULTS (Final Baseline)')
print('=' * 60)

print(f'\n{"Model":<25s} {"AUROC":>8s} {"AUPRC":>8s} {"Brier":>8s} {"Acc":>8s}')
print('-' * 60)

for name, model in models.items():
    y_prob = model.predict_proba(X_test_scaled)[:, 1]
    m = evaluate_model(name, y_test, y_prob)
    print(f'{name:<25s} {m["AUROC"]:>8.4f} {m["AUPRC"]:>8.4f} {m["Brier"]:>8.4f} {m["Acc"]:>8.4f}')

m_test = evaluate_model('Ensemble', y_test, ensemble_probs_test)
print(f'{"Ensemble (GB+RF)":<25s} {m_test["AUROC"]:>8.4f} {m_test["AUPRC"]:>8.4f} {m_test["Brier"]:>8.4f} {m_test["Acc"]:>8.4f}')

In [ ]:
# 10E: Visualization - ROC + PR curves + Feature importance
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# ROC curve
for name, model in models.items():
    y_prob = model.predict_proba(X_test_scaled)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_prob)
    auc = roc_auc_score(y_test, y_prob)
    axes[0].plot(fpr, tpr, label=f'{name} (AUC={auc:.3f})', linewidth=2)

fpr_e, tpr_e, _ = roc_curve(y_test, ensemble_probs_test)
auc_e = roc_auc_score(y_test, ensemble_probs_test)
axes[0].plot(fpr_e, tpr_e, label=f'Ensemble (AUC={auc_e:.3f})', linewidth=2, linestyle='--')
axes[0].plot([0, 1], [0, 1], 'k--', alpha=0.3)
axes[0].set_xlabel('False Positive Rate')
axes[0].set_ylabel('True Positive Rate')
axes[0].set_title('ROC Curve (Test Set)')
axes[0].legend()

# PR curve
for name, model in models.items():
    y_prob = model.predict_proba(X_test_scaled)[:, 1]
    prec, rec, _ = precision_recall_curve(y_test, y_prob)
    ap = average_precision_score(y_test, y_prob)
    axes[1].plot(rec, prec, label=f'{name} (AP={ap:.3f})', linewidth=2)

prec_e, rec_e, _ = precision_recall_curve(y_test, ensemble_probs_test)
ap_e = average_precision_score(y_test, ensemble_probs_test)
axes[1].plot(rec_e, prec_e, label=f'Ensemble (AP={ap_e:.3f})', linewidth=2, linestyle='--')
axes[1].set_xlabel('Recall')
axes[1].set_ylabel('Precision')
axes[1].set_title('Precision-Recall Curve (Test Set)')
axes[1].legend()

# Feature importance (from GB model)
importances = gb.feature_importances_
idx = np.argsort(importances)[::-1]
axes[2].barh(range(len(used_features)), importances[idx], color='steelblue', edgecolor='white')
axes[2].set_yticks(range(len(used_features)))
axes[2].set_yticklabels([used_features[i] for i in idx])
axes[2].set_xlabel('Feature Importance')
axes[2].set_title('GradientBoosting Feature Importance')
axes[2].invert_yaxis()

plt.suptitle('RQ1 Baseline: Monolithic Model Performance', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# 10F: Save baseline results
baseline_results = {
    'experiment': 'RQ1_monolithic_baseline',
    'date': datetime.now().isoformat(),
    'dataset': {
        'train_size': len(train),
        'val_size': len(val),
        'test_size': len(test),
        'features': used_features,
        'n_features': len(used_features),
    },
    'results': results,
    'test_results': {
        'ensemble_auroc': float(auc_e),
        'ensemble_auprc': float(ap_e),
        'ensemble_brier': float(m_test['Brier']),
        'ensemble_accuracy': float(m_test['Accuracy']),
    }
}

results_dir = PROJECT_ROOT / 'outputs' / 'research'
results_dir.mkdir(parents=True, exist_ok=True)

with open(results_dir / 'rq1_monolithic_baseline.json', 'w') as f:
    json.dump(baseline_results, f, indent=2, default=str)

# Save models
with open(results_dir / 'baseline_gb.pkl', 'wb') as f:
    pickle.dump(gb, f)
with open(results_dir / 'baseline_rf.pkl', 'wb') as f:
    pickle.dump(rf, f)
with open(results_dir / 'baseline_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
with open(results_dir / 'baseline_imputer.pkl', 'wb') as f:
    pickle.dump(imputer, f)

print(f'Baseline results saved to {results_dir}/')
print(f'\nNext steps:')
print(f'  1. Notebook 04: Modality dropout experiments (RQ1)')
print(f'  2. Notebook 05: Agentic model comparison (RQ1)')
print(f'  3. Notebook 06: Communication protocol experiments (RQ2)')

---
## Summary

### Data Audit Results
| Dataset | Records | Features | Coverage | Quality |
|---------|---------|----------|----------|---------|
| ml_ready_data.csv | 53K | 10 | 2020-2025 | Check water_temp zeros |
| in_situ (LUBW) | 49K | 23 | 2025 | Rich, detailed |
| satellite | 8.7K | 24 | 2020-2025 | Spectral indices |
| lake_json | varies | bloom labels | 2020-2025 | Has bloom_probability! |

### Baseline Established
- Monolithic GB + RF ensemble trained on in-situ + satellite features
- Proper temporal train/val/test split
- Research-standard metrics: AUROC, AUPRC, Brier, Accuracy

### Next Notebooks
- **04**: Modality dropout (remove satellite/insitu features, measure degradation)
- **05**: Agentic model (run same data through SWIM agents, compare)
- **06**: Communication protocol experiments (RQ2)